### '아파트 청약'기사 크롤링 함수
- 내용에 '아파트 청약' 키워드가 포함된 기사 크롤링
    - 기간: 2020~2024
    - 합계: 410개
        - 2020~2022: 각 100개
        - 2023: 29개
        - 2024: 81개

In [2]:
import urllib.parse
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import random

def generate_news_url(query: str, start: int, start_date: str, end_date: str) -> str:
    """네이버 뉴스 검색 URL 생성"""
    base_url = "https://search.naver.com/search.naver"
    params = {
        "where": "news",
        "query": query,
        "sm": "tab_opt",
        "sort": "0",  # 0: 관련도순 (무작위성 증가)
        "start": start,
        "pd": "3",  # 기간 설정 사용
        "ds": start_date,  # 시작일
        "de": end_date,  # 종료일
        "nso": f"so:r,p:from{start_date.replace('.', '')}to{end_date.replace('.', '')},a:all"
    }
    return base_url + "?" + urllib.parse.urlencode(params, quote_via=urllib.parse.quote)

def crawl_naver_news(query: str, year: int, max_articles: int = 100) -> list:
    """네이버 뉴스에서 특정 키워드와 연도별 기사 크롤링"""
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    articles = []
    start_date = f"{year}.01.01"
    end_date = f"{year}.12.31"

    while len(articles) < max_articles:
        start = random.randint(1, 1000)  # 무작위 시작 페이지
        try:
            search_url = generate_news_url(query, start, start_date, end_date)
            response = requests.get(search_url, headers=headers)
            response.raise_for_status()

            soup = BeautifulSoup(response.text, 'html.parser')
            news_links = [
                a['href'] for a in soup.select('a.info')
                if '네이버뉴스' in a.text and 'news.naver.com' in a['href']
            ]

            if not news_links:
                print(f"{year}년에는 더 이상 기사가 없습니다.")
                break

            for article_url in news_links:
                if len(articles) >= max_articles:
                    break

                try:
                    article_res = requests.get(article_url, headers=headers, timeout=5)
                    article_res.raise_for_status()
                    article_soup = BeautifulSoup(article_res.text, 'html.parser')

                    content_elem = article_soup.select_one('#newsct_article, #dic_area')
                    if not content_elem:
                        continue

                    content = content_elem.get_text(strip=True)
                    if "아파트 청약" not in content:
                        continue

                    title_elem = article_soup.select_one('#title_area')
                    date_elem = article_soup.select_one('.media_end_head_info_datestamp_time')

                    if title_elem and date_elem:
                        articles.append({
                            "title": title_elem.get_text(strip=True),
                            "date": date_elem['data-date-time'],
                            "content": content,
                            "url": article_url,
                            "year": year
                        })

                except Exception as e:
                    print(f"기사 처리 실패 ({article_url}): {str(e)}")

            time.sleep(random.uniform(1, 3))  # 무작위 대기 시간

        except Exception as e:
            print(f"페이지 처리 실패 ({search_url}): {str(e)}")
            time.sleep(random.uniform(5, 10))  # 오류 시 더 긴 대기 시간

    return articles

if __name__ == "__main__":
    query = "아파트 청약"

    print(f"{query} 관련 뉴스 크롤링 시작...")

    all_news_data = []

    for year in range(2020, 2025):  # 2020년부터 2024년까지 반복
        print(f"\n===== {year}년 크롤링 시작 =====")
        yearly_news_data = crawl_naver_news(query, year, max_articles=100)
        all_news_data.extend(yearly_news_data)
        print(f"{year}년 크롤링 완료: {len(yearly_news_data)}개의 기사 수집")

    print(f"\n총 {len(all_news_data)}개의 기사가 수집되었습니다.")

    # 결과 출력 및 저장
    df = pd.DataFrame(all_news_data)
    df.to_csv('202024_아파트청약_뉴스_연도별_크롤링.csv', index=False, encoding='utf-8-sig')

    print("크롤링 결과가 '2020_2024_아파트청약_뉴스_연도별_크롤링.csv' 파일로 저장되었습니다.")

아파트 청약 관련 뉴스 크롤링 시작...

===== 2020년 크롤링 시작 =====
2020년 크롤링 완료: 100개의 기사 수집

===== 2021년 크롤링 시작 =====
2021년 크롤링 완료: 100개의 기사 수집

===== 2022년 크롤링 시작 =====
2022년 크롤링 완료: 100개의 기사 수집

===== 2023년 크롤링 시작 =====
2023년에는 더 이상 기사가 없습니다.
2023년 크롤링 완료: 29개의 기사 수집

===== 2024년 크롤링 시작 =====
2024년에는 더 이상 기사가 없습니다.
2024년 크롤링 완료: 81개의 기사 수집

총 410개의 기사가 수집되었습니다.
크롤링 결과가 '2020_2024_아파트청약_뉴스_연도별_크롤링.csv' 파일로 저장되었습니다.
